### Konstantina Andronikou

## Comparison of Names as a Robustness test

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [3]:
from checklist.pred_wrapper import PredictorWrapper

### Western Names

In [4]:
#removing all warnings 
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Maria Smith likes to eat pizza")

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Maria Smith likes to eat pizza"]
pred = predict_and_conf(d)

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name'],meta['last_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:

editor = Editor()

# create examples
t = editor.template('{first_name} {last_name} likes to eat pizza', meta=True, nsamples=1000, remove_duplicates = True)

# for k, v in t.items():
#     print(k,v)

In [12]:
with open('Dataset_BiLSTM/Western_names.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Western_names.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
#         print(case,exp)

Predicting 999 examples
Test cases:      999
Fails (rate):    26 (2.6%)

Example fails:
[ARGM-DIS: Christine] [ARG0: Adams] [V: likes] [ARG1: to eat pizza]
----
[ARGM-ADV: Benjamin] [ARG0: Stewart] [V: likes] [ARG1: to eat pizza]
----
[ARG0: Julia] Lewis [V: likes] [ARG1: to eat pizza]
----


In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Western_names.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Non-Western Names 

In [15]:
first = [x.split()[0] for x in editor.lexicons.male_from.Turkey +  editor.lexicons.female_from.Turkey]
last = [x.split()[0] for x in editor.lexicons.last_from.Turkey]

In [16]:
with open('Dataset_BiLSTM/Non-western_names.txt', 'w') as f:
    print(t.data, file = f)

In [17]:
t = editor.template("{first_name} {last_name} likes to eat pizza.", first_name=first, last_name=last, meta=True, nsamples=1000)
test = MFT(**t, expect=expect_arg0)
test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Non-Western_names.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    29 (2.9%)

Example fails:
[ARG2: Halil] [V: Can] [ARG1: likes to eat pizza] .
----
Hasan [ARG0: Polat] [V: likes] [ARG1: to eat pizza] .
----
[V: Can] [ARG1: Doğan] likes to eat pizza .
----


In [18]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Non-Western_names.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 